In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [9]:
truth = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/sen_spe_files_07_09_2023/VCC_input_files/Truthset_CHROM_POS_REF_ALT.tsv', sep = '\t')
truth

,CHROM,POS,REF,ALT
0,chr1,65395,T,A
1,chr1,65403,G,C
2,chr1,65414,C,G
3,chr1,65434,G,A
4,chr1,65445,C,A
...,...,...,...,...
37236450,chrY,57194336,T,C
37236451,chrY,57194337,T,C
37236452,chrY,57194340,T,A
37236453,chrY,57194341,T,C


In [10]:
import pandas as pd
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/sen_spe_files_07_09_2023/VCC_input_files/12652707_BCFTOOL.vcf', comment='#', sep='\t', header=None, low_memory=False, encoding='latin-1')
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf['AD'] = vcf['SAMPLE'].str.split(':').str[6]
vcf['RD'] = vcf['AD'].str.split(',').str[0].astype(int)
vcf['A_D'] = vcf['AD'].str.split(',').str[1].astype(int)
vcf['VAF'] = vcf['A_D'] / (vcf['RD'] + vcf['A_D'])
vcf['DP'] = vcf['INFO'].str.extract(r'DP=(\d+)')[0].fillna('0').astype(int)
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,AD,RD,A_D,VAF,DP
0,chr1,817514,rs3131971,T,C,85.9373,.,"DP=244;ADF=122,82;ADR=3,19;AD=125,101;SCR=0;VD...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:114,17,0:226:43:122,82:3,19:125,101:0","125,101",125,101,0.446903,244
1,chr1,930939,rs9988021,G,A,225.4170,.,"DP=71;ADF=0,54;ADR=0,11;AD=0,65;SCR=0;VDB=0.49...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:255,196,0:65:0:0,54:0,11:0,65:0","0,65",0,65,1.000000,71
2,chr1,935954,rs4072383,G,T,224.4170,.,"DP=19;ADF=0,2;ADR=0,11;AD=0,13;SCR=0;VDB=0.087...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:254,39,0:13:0:0,2:0,11:0,13:0","0,13",0,13,1.000000,19
3,chr1,941119,rs4372192,A,G,225.4170,.,"DP=23;ADF=0,17;ADR=0,3;AD=0,20;SCR=0;VDB=0.280...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:255,60,0:20:0:0,17:0,3:0,20:0","0,20",0,20,1.000000,23
4,chr1,944858,rs3748592,A,G,225.4170,.,"DP=20;ADF=0,11;ADR=0,7;AD=0,18;SCR=0;VDB=0.165...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:255,54,0:18:0:0,11:0,7:0,18:0","0,18",0,18,1.000000,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77296,chrY,11155547,rs62599748,T,A,121.4150,.,"DP=53;ADF=0,40;ADR=0,5;AD=0,45;SCR=0;VDB=0.166...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:151,135,0:45:0:0,40:0,5:0,45:0","0,45",0,45,1.000000,53
77297,chrY,12340004,rs377316799,G,T,200.4160,.,"DP=52;ADF=0,24;ADR=0,27;AD=0,51;SCR=0;VDB=0.17...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:230,154,0:51:0:0,24:0,27:0,51:0","0,51",0,51,1.000000,52
77298,chrY,12914512,rs2032624,C,A,225.4170,.,"DP=33;ADF=0,24;ADR=0,6;AD=0,30;SCR=0;VDB=0.299...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:255,90,0:30:0:0,24:0,6:0,30:0","0,30",0,30,1.000000,33
77299,chrY,13250864,rs376039285,C,T,189.4160,.,"DP=28;ADF=0,25;ADR=0,0;AD=0,25;SCR=0;VDB=0.189...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:219,75,0:25:0:0,25:0,0:0,25:0","0,25",0,25,1.000000,28


In [13]:
intersection_ab = pd.merge(vcf, truth, on=['CHROM', 'POS', 'REF', 'ALT'])
intersection_ab

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,AD,RD,A_D,VAF,DP
0,chr1,946247,rs2272757,G,A,225.4170,.,"DP=52;ADF=0,23;ADR=0,24;AD=0,47;SCR=0;VDB=0.03...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:255,141,0:47:0:0,23:0,24:0,47:0","0,47",0,47,1.000000,52
1,chr1,948711,rs7522415,C,G,40.5469,.,"DP=15;ADF=0,1;ADR=10,4;AD=10,5;SCR=0;VDB=0.240...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"0/1:74,0,148:15:0:0,1:10,4:10,5:0","10,5",10,5,0.333333,15
2,chr1,948721,rs4970455,A,C,58.1845,.,"DP=10;ADF=1,0;ADR=4,4;AD=5,4;SCR=0;VDB=0.01758...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"0/1:91,0,52:9:0:1,0:4,4:5,4:0","5,4",5,4,0.444444,10
3,chr1,951437,rs10465241,C,T,104.4150,.,"DP=7;ADF=0,0;ADR=0,5;AD=0,5;SCR=0;VDB=0.203629...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:134,15,0:5:0:0,0:0,5:0,5:0","0,5",0,5,1.000000,7
4,chr1,953779,rs13302945,A,C,225.4170,.,"DP=32;ADF=0,14;ADR=0,14;AD=0,28;SCR=0;VDB=0.00...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:255,84,0:28:0:0,14:0,14:0,28:0","0,28",0,28,1.000000,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29174,chrY,1425778,.,G,A,37.4152,.,"DP=86;ADF=0,13;ADR=0,66;AD=0,79;SCR=0;VDB=0.85...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:67,238,0:79:0:0,13:0,66:0,79:0","0,79",0,79,1.000000,86
29175,chrY,6872663,rs34551924,A,G,214.4170,.,"DP=20;ADF=0,1;ADR=0,18;AD=0,19;SCR=0;VDB=0.031...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:244,57,0:19:0:0,1:0,18:0,19:0","0,19",0,19,1.000000,20
29176,chrY,12914512,rs2032624,C,A,225.4170,.,"DP=33;ADF=0,24;ADR=0,6;AD=0,30;SCR=0;VDB=0.299...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:255,90,0:30:0:0,24:0,6:0,30:0","0,30",0,30,1.000000,33
29177,chrY,13250864,rs376039285,C,T,189.4160,.,"DP=28;ADF=0,25;ADR=0,0;AD=0,25;SCR=0;VDB=0.189...",GT:PL:DP:SP:ADF:ADR:AD:SCR,"1/1:219,75,0:25:0:0,25:0,0:0,25:0","0,25",0,25,1.000000,28


In [6]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/MODY_new_05_10_2023/All_5samples.xlsx')
df['CHROM'] = 'chr' + df['Chr:Pos'].str.split(':').str[0]
df['POS'] = df['Chr:Pos'].str.split(':').str[1].astype(int)
df['REF'] = df['Ref/Alt'].str.split('/').str[0]
df['ALT'] = df['Ref/Alt'].str.split('/').str[1]
columns_to_check = ['CHROM', 'POS', 'REF', 'ALT']
# Use drop_duplicates to remove duplicated rows based on the specified columns
df = df.drop_duplicates(subset=columns_to_check)
df

,Chr:Pos,Ref/Alt,Gene Name,Gene Inheritance,Entrez Gene ID,Transcript Name,Transcript Strand,HGVS cDot,HGVS pDot,Sequence Ontology,Other Transcript Effects,ACMG Classification Criteria,ACMG Classification Criteria Description,Classification,Auto Classification,Previous Classification Count,Previous Classification,Last Classification Date,Max Sub Population Freq Group Name,Max MAF,Max Allele Number,Max Allele Count,CHROM,POS,REF,ALT
0,1:69511,A/G,OR4F5,Default (Recessive),79501,NM_001005484.2,+,NM_001005484.2:c.484A>G,NP_001005484.2:p.Thr162Ala,missense_variant,Single Transcript Gene,"BA1,BP4",OR4F5:Allele frequency is above 0.01 recessive...,Benign,Benign,NaN,NaN,NaN,Annotated gnomAD East Asian,0.999461,16698,16689,chr1,69511,A,G
1,1:817514,T/C,FAM87B,Default (Recessive),400728,NR_103536.1,+,NR_103536.1:n.144T>C,NaN,non_coding_exon_variant,Single Transcript Gene,"BA1,BS2,BP7",FAM87B:Allele frequency is above 0.01 recessiv...,Benign,Benign,NaN,NaN,NaN,Annotated 1kG All,0.753195,5008,3772,chr1,817514,T,C
2,1:826893,G/A,LINC00115,Default (Recessive),79854,NR_024321.1,-,NR_024321.1:n.630C>T,NaN,non_coding_exon_variant,Single Transcript Gene,"BA1,BS2,BP7",LINC00115:Allele frequency is above 0.01 reces...,Benign,Benign,NaN,NaN,NaN,Annotated gnomAD East Asian,0.896992,10106,9065,chr1,826893,G,A
3,1:827209,G/C,LINC00115,Default (Recessive),79854,NR_024321.1,-,NR_024321.1:n.314C>G,NaN,non_coding_exon_variant,Single Transcript Gene,"BA1,BS2,BP7",LINC00115:Allele frequency is above 0.01 reces...,Benign,Benign,NaN,NaN,NaN,Annotated gnomAD East Asian,0.894115,7952,7110,chr1,827209,G,C
4,1:827212,C/G,LINC00115,Default (Recessive),79854,NR_024321.1,-,NR_024321.1:n.311G>C,NaN,non_coding_exon_variant,Single Transcript Gene,"BA1,BS2,BP7",LINC00115:Allele frequency is above 0.01 reces...,Benign,Benign,NaN,NaN,NaN,Annotated gnomAD East Asian,0.894327,7826,6999,chr1,827212,C,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481215,X:154641461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chrX,154641461,NaN,NaN
481219,X:154773313,A/T,DKC1,Recessive,1736,NM_001363.5,+,NM_001363.5:c.1155+64A>T,NaN,intron_variant,Other Transcripts Have Same Effect,"PM2,BP7",DKC1:Variant is missing from all sub populatio...,VUS/Conflicting,VUS/Conflicting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chrX,154773313,A,T
481223,X:154785208,-/C,MPP1,Default (Recessive),4354,NM_002436.4,-,NM_002436.4:c.678-51_678-50insG,NaN,intron_variant,Other Transcripts Have Same Effect,"BA1,BS2,BP7",MPP1:Allele frequency is above 0.01 recessive ...,Benign,Benign,NaN,NaN,NaN,Annotated 1kG All,0.0492715,3775,186,chrX,154785208,-,C
481224,X:154785208,A/C,MPP1,Default (Recessive),4354,NM_002436.4,-,NM_002436.4:c.678-51T>G,NaN,intron_variant,Other Transcripts Have Same Effect,"BA1,BS2,BP7",MPP1:Allele frequency is above 0.01 recessive ...,Benign,Benign,NaN,NaN,NaN,Annotated 1kG All,0.132715,3775,501,chrX,154785208,A,C


In [7]:
df.to_excel(r'C:/Users/s/GenepoweRx_Madhu/Desktop/all_5samples_data.xlsx', index=False)

In [7]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/covered_not_covered_file.xlsx')
df

,Variant,CHROM,POS,RET,ALT,Covered/Not_Covered
0,rs75527207,chr7,117587806.0,G,A,Covered
1,rs4149056,chr12,21178615.0,T,C,Covered
2,rs1799971,chr6,154039662.0,A,G,Covered
3,rs141033578,chr7,117606695.0,C,G,Covered
4,rs78769542,chr7,117611650.0,G,C,Covered
...,...,...,...,...,...,...
2299,rs351776,chr8,28333789.0,A,T,Not_Covered
2300,rs78132896,chr6,93420339.0,T,C,Not_Covered
2301,rs1059513,chr12,57095926.0,T,C,Not_Covered
2302,rs12885713,chr14,90397013.0,C,G,Not_Covered


In [8]:
data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/mapped_pmid.xlsx')
data

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant,Haplotypes,Gene,Level of Evidence,Level Override,Level Modifiers,Score,Phenotype Category,PMID,PMID Count,Evidence Count,Drug(s),Phenotype(s),Latest History Date (YYYY-MM-DD),URL,Specialty Population
0,981755803,AA,Patients with the rs75527207 AA genotype (two ...,NaN,rs75527207,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric
1,981755803,AG,Patients with the rs75527207 AG genotype (one ...,NaN,rs75527207,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric
2,981755803,GG,Patients with the rs75527207 GG genotype (do n...,NaN,rs75527207,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric
3,1449311190,CC,Patients with the CC genotype and Precursor Ce...,NaN,rs4149056,NaN,SLCO1B1,3,NaN,Tier 1 VIP,2.000,Dosage,29683944,1,1,mercaptopurine;methotrexate,Precursor Cell Lymphoblastic Leukemia-Lymphoma,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,Pediatric
4,1449311190,CT,Patients with the CT genotype and Precursor Ce...,NaN,rs4149056,NaN,SLCO1B1,3,NaN,Tier 1 VIP,2.000,Dosage,29683944,1,1,mercaptopurine;methotrexate,Precursor Cell Lymphoblastic Leukemia-Lymphoma,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,Pediatric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15648,1449000354,*64,Patients with the CYP2D6*64 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
15649,1449000354,*65,Patients with the CYP2D6*65 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
15650,1449000354,*70,Patients with the CYP2D6*70 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
15651,1449000354,*71,Patients with the CYP2D6*71 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN


In [9]:
merged_df = pd.merge(data, df, on = 'Variant', how = 'left', sort=False)
merged_df

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant,Haplotypes,Gene,Level of Evidence,Level Override,Level Modifiers,Score,Phenotype Category,PMID,PMID Count,Evidence Count,Drug(s),Phenotype(s),Latest History Date (YYYY-MM-DD),URL,Specialty Population,CHROM,POS,RET,ALT,Covered/Not_Covered
0,981755803,AA,Patients with the rs75527207 AA genotype (two ...,NaN,rs75527207,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric,chr7,117587806.0,G,A,Covered
1,981755803,AG,Patients with the rs75527207 AG genotype (one ...,NaN,rs75527207,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric,chr7,117587806.0,G,A,Covered
2,981755803,GG,Patients with the rs75527207 GG genotype (do n...,NaN,rs75527207,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric,chr7,117587806.0,G,A,Covered
3,1449311190,CC,Patients with the CC genotype and Precursor Ce...,NaN,rs4149056,NaN,SLCO1B1,3,NaN,Tier 1 VIP,2.000,Dosage,29683944,1,1,mercaptopurine;methotrexate,Precursor Cell Lymphoblastic Leukemia-Lymphoma,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,Pediatric,chr12,21178615.0,T,C,Covered
4,1449311190,CT,Patients with the CT genotype and Precursor Ce...,NaN,rs4149056,NaN,SLCO1B1,3,NaN,Tier 1 VIP,2.000,Dosage,29683944,1,1,mercaptopurine;methotrexate,Precursor Cell Lymphoblastic Leukemia-Lymphoma,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,Pediatric,chr12,21178615.0,T,C,Covered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15648,1449000354,*64,Patients with the CYP2D6*64 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN,NaN,NaN,NaN,NaN,NaN
15649,1449000354,*65,Patients with the CYP2D6*65 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN,NaN,NaN,NaN,NaN,NaN
15650,1449000354,*70,Patients with the CYP2D6*70 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN,NaN,NaN,NaN,NaN,NaN
15651,1449000354,*71,Patients with the CYP2D6*71 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
merged_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/anticancer_pgx/MAIN_mapped_rsid_covered_data.xlsx', index = False)

In [18]:
df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/anticancer_pgx/MAIN_mapped_rsid_covered_data.xlsx')
df_1

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant,CHROM,POS,RET,ALT,Covered/Not_Covered,Haplotypes,Gene,Level of Evidence,Level Override,Level Modifiers,Score,Phenotype Category,PMID,PMID Count,Evidence Count,Drug(s),Phenotype(s),Latest History Date (YYYY-MM-DD),URL,Specialty Population
0,981755803,AA,Patients with the rs75527207 AA genotype (two ...,NaN,rs75527207,chr7,117587806.0,G,A,Covered,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric
1,981755803,AG,Patients with the rs75527207 AG genotype (one ...,NaN,rs75527207,chr7,117587806.0,G,A,Covered,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric
2,981755803,GG,Patients with the rs75527207 GG genotype (do n...,NaN,rs75527207,chr7,117587806.0,G,A,Covered,NaN,CFTR,1A,NaN,Rare Variant; Tier 1 VIP,234.875,Efficacy,"21083385, 22047557, 23590265, 23757361, 238913...",28,30,ivacaftor,Cystic Fibrosis,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/98...,Pediatric
3,1449311190,CC,Patients with the CC genotype and Precursor Ce...,NaN,rs4149056,chr12,21178615.0,T,C,Covered,NaN,SLCO1B1,3,NaN,Tier 1 VIP,2.000,Dosage,29683944,1,1,mercaptopurine;methotrexate,Precursor Cell Lymphoblastic Leukemia-Lymphoma,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,Pediatric
4,1449311190,CT,Patients with the CT genotype and Precursor Ce...,NaN,rs4149056,chr12,21178615.0,T,C,Covered,NaN,SLCO1B1,3,NaN,Tier 1 VIP,2.000,Dosage,29683944,1,1,mercaptopurine;methotrexate,Precursor Cell Lymphoblastic Leukemia-Lymphoma,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,Pediatric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15648,1449000354,*64,Patients with the CYP2D6*64 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
15649,1449000354,*65,Patients with the CYP2D6*65 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
15650,1449000354,*70,Patients with the CYP2D6*70 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
15651,1449000354,*71,Patients with the CYP2D6*71 allele may have de...,Uncertain function,"CYP2D6*1, CYP2D6*7, CYP2D6*9, CYP2D6*10, CYP2D...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,3,NaN,Tier 1 VIP,0.000,Metabolism/PK,24647041,1,6,n-desmethyltamoxifen,NaN,2023-08-31,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN


In [19]:
df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Drugs_brest_cancer.xlsx')
df_2['Drug(s)'] = df_2['Drug(s)'].str.lower()
df_2

,Drug(s)
0,cyclophosphamide;doxorubicin;fluorouracil
1,cyclophosphamide;doxorubicin
2,carboplatin;docetaxel;trastuzumab
3,cyclophosphamide;epirubicin
4,cyclophosphamide;epirubicin;fluorouracil
5,tamoxifen
6,anthracyclines and related substances;taxanes
7,capecitabine;docetaxel
8,paclitaxel
9,doxorubicin;doxorubicinol


In [20]:
merged_df2 = pd.merge(df_1, df_2, on = 'Drug(s)', how = 'inner', sort = False)
merged_df2

,Clinical Annotation ID,Genotype/Allele,Annotation Text,Allele Function,Variant,CHROM,POS,RET,ALT,Covered/Not_Covered,Haplotypes,Gene,Level of Evidence,Level Override,Level Modifiers,Score,Phenotype Category,PMID,PMID Count,Evidence Count,Drug(s),Phenotype(s),Latest History Date (YYYY-MM-DD),URL,Specialty Population
0,1451285240,*1,The CYP2D6*1 allele is assigned as a normal fu...,Normal function,"CYP2D6*1, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,1A,NaN,Tier 1 VIP,226.5625,Efficacy,"24329190, 23100173, 23842856, 25091503, 223956...",56,143,tamoxifen,Breast Neoplasms,2021-04-29,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
1,1451285240,*2,The CYP2D6*2 allele is assigned as a normal fu...,Normal function,"CYP2D6*1, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,1A,NaN,Tier 1 VIP,226.5625,Efficacy,"24329190, 23100173, 23842856, 25091503, 223956...",56,143,tamoxifen,Breast Neoplasms,2021-04-29,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
2,1451285240,*3,The CYP2D6*3 allele is assigned as a no functi...,No function,"CYP2D6*1, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,1A,NaN,Tier 1 VIP,226.5625,Efficacy,"24329190, 23100173, 23842856, 25091503, 223956...",56,143,tamoxifen,Breast Neoplasms,2021-04-29,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
3,1451285240,*4,The CYP2D6*4 allele is assigned as a no functi...,No function,"CYP2D6*1, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,1A,NaN,Tier 1 VIP,226.5625,Efficacy,"24329190, 23100173, 23842856, 25091503, 223956...",56,143,tamoxifen,Breast Neoplasms,2021-04-29,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
4,1451285240,*5,The CYP2D6*5 allele is assigned as a no functi...,No function,"CYP2D6*1, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6...",NaN,NaN,NaN,NaN,NaN,NaN,CYP2D6,1A,NaN,Tier 1 VIP,226.5625,Efficacy,"24329190, 23100173, 23842856, 25091503, 223956...",56,143,tamoxifen,Breast Neoplasms,2021-04-29,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,1448112147,CT,Patients with the CT genotype and hormone inse...,NaN,rs4149056,chr12,21178615.0,T,C,Covered,NaN,SLCO1B1,3,NaN,Tier 1 VIP,1.5000,Toxicity,27234217,1,1,cyclophosphamide;docetaxel;doxorubicin;epirubi...,Breast Neoplasms,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
709,1448112147,TT,Patients with the TT genotype and hormone inse...,NaN,rs4149056,chr12,21178615.0,T,C,Covered,NaN,SLCO1B1,3,NaN,Tier 1 VIP,1.5000,Toxicity,27234217,1,1,cyclophosphamide;docetaxel;doxorubicin;epirubi...,Breast Neoplasms,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
710,1447963611,AA,Patients with the AA genotype and node-positiv...,NaN,rs351855,chr5,177093242.0,G,A,Covered,NaN,FGFR4,3,NaN,NaN,1.7500,Efficacy,16822847,1,1,cyclophosphamide;fluorouracil;methotrexate,Breast Neoplasms,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN
711,1447963611,AG,Patients with the AG genotype and node-positiv...,NaN,rs351855,chr5,177093242.0,G,A,Covered,NaN,FGFR4,3,NaN,NaN,1.7500,Efficacy,16822847,1,1,cyclophosphamide;fluorouracil;methotrexate,Breast Neoplasms,2021-03-24,https://www.pharmgkb.org/clinicalAnnotation/14...,NaN


In [30]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHHSPTGPPGX9_final.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR']]
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR
0,chr1,14677,rs201327123,G,A,.,PASS,ADP=65;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,63,65,65,46,19,29.23%,4.0861E-7,40,6,18,1
1,chr1,14907,rs6682375,A,G,.,PASS,ADP=87;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,132,87,87,50,37,42.53%,5.4067E-14,30,20,28,9
2,chr1,14930,rs6682385,A,G,.,PASS,"ADP=89;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5178,0....",0/1,163,89,89,45,44,49.44%,4.4109E-17,27,18,32,12
3,chr1,15118,rs11580262,A,G,.,PASS,ADP=41;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=WASH...,0/1,47,41,41,27,14,34.15%,1.6028E-5,9,18,2,12
4,chr1,15820,rs2691315,G,T,.,PASS,"ADP=17;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5895,.,...",0/1,43,17,17,6,11,64.71%,4.3258E-5,5,1,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80173,chrY,57209899,.,C,T,.,PASS,ADP=10;WT=0;HET=0;HOM=1;NC=0;CSQ=T|non_coding_...,1/1,34,10,10,2,8,80%,3.5723E-4,2,0,6,2
80174,chrY,57210867,.,T,C,.,PASS,ADP=13;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,21,13,13,7,6,46.15%,7.4534E-3,2,5,1,5
80175,chrY,57211683,.,T,C,.,PASS,ADP=29;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,30,29,29,20,9,31.03%,9.4048E-4,17,3,4,5
80176,chrY,57211949,.,T,C,.,PASS,ADP=17;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,0/1,33,17,17,8,9,52.94%,4.6348E-4,1,7,2,7


In [31]:
vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf['GT'] = vcf['GT'].astype(str)
vcf = vcf[['rsID', 'Zygosity']]
vcf

,rsID,Zygosity
0,rs201327123,Heterozygous
1,rs6682375,Heterozygous
2,rs6682385,Heterozygous
3,rs11580262,Heterozygous
4,rs2691315,Heterozygous
...,...,...
80173,.,Homozygous
80174,.,Heterozygous
80175,.,Heterozygous
80176,.,Heterozygous


In [32]:
df_3 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/brest_cancer_variants.xlsx')
df_3

,rsID
0,rs717620
1,rs2740574
2,rs1804645
3,rs4646
4,rs3732218
...,...
61,rs1061170
62,rs5370
63,rs2073618
64,rs10046


In [33]:
merged_df3 = pd.merge(df_3, vcf, on = 'rsID', how = 'left', sort=False)
merged_df3['Zygosity'].fillna('Wild', inplace=True)
merged_df3

,rsID,Zygosity
0,rs717620,Wild
1,rs2740574,Homozygous
2,rs1804645,Wild
3,rs4646,Heterozygous
4,rs3732218,Wild
...,...,...
61,rs1061170,Heterozygous
62,rs5370,Heterozygous
63,rs2073618,Heterozygous
64,rs10046,Heterozygous


In [35]:
merged_df3.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/anticancer_pgx/KHHSPTGPPGX9_rsID_data.xlsx', index=False)